In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent)) #katalog główny repo do sys.path

In [2]:
from src.data_loading import load_data, merge_data

transactions_df, customers_df, articles_df = load_data()
df = merge_data(transactions_df, articles_df)


Data path: C:\Users\wszyl\Desktop\CV\Licencjat\CustomerChurnPrediction\Customer-Churn-Prediction\data\raw
